# Libraries

In [1]:
from _ctypes import LoadLibrary as _dlopen
_dlopen("D:\\Sangwons_Room\\00_SoftWares\\Anaconda\\envs\\torch\\Library\\bin\\geos.dll", 0)

# nuscenes-devkit
from nuscenes.nuscenes import NuScenes

# nuscenes-map expansion
from utils.custom_map_api_expansion import CustomNuScenesMap

# nuscenes-devkit-utils
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
from nuscenes.utils.geometry_utils import view_points, transform_matrix

from pyquaternion import Quaternion


from functools import reduce
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import tqdm

from torch.utils.data import Dataset, DataLoader
%matplotlib inline

# Set  Parameters

In [2]:
# locations = ['singapore-onenorth', 'singapore-hollandvillage', 'singapore-queenstown', 'boston-seaport']
version = 'v1.0-mini'
dataroot = './data/sets/nuscenes'

# Load Informations

In [3]:
nusc = NuScenes(version=version, dataroot=dataroot, verbose=True)
nusc_location = dict()
for loc in locations:
    nusc_location[loc] = CustomNuScenesMap(dataroot=dataroot, map_name=loc)

Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.684 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


# Check Maps

In [4]:
# for location in locations:
#     nusc_map = nusc_location[location]
#     fig, ax = nusc_map.render_layers(nusc_map.non_geometric_layers, figsize=1)
#     ax.set_title(f"{location}, {nusc_map.get_size()}")
# plt.show()

In [6]:
# nusc_map = nusc_location['singapore-hollandvillage']

nusc_map = nusc_location['singapore-onenorth']

nusc_map.get_closest_layers(nusc_map.non_geometric_layers, (700.0, 1900.0))

{'token': '00af3709-6b7b-4f63-9b28-c6565eeeac66', 'polygon_token': 'db425bb0-0403-4380-8571-4443fcba45af', 'is_intersection': False, 'drivable_area_token': 'c3e28556-b711-4581-9970-b66166fb907d', 'exterior_node_tokens': ['f4e7f647-844a-4f01-b5fc-3d3b360643b7', 'fbc535f8-65d4-412d-a86c-d60a31bce020', '6d4cd4fd-b06b-4085-91d6-081ccfc66a47', '51c387f0-5398-4c4e-bad9-ae5e828a9a23', '7e233b3e-2e5d-43cc-9ebe-fb30eace641d', '3391647a-b3f9-4631-97d1-43674ac4be22', '9771a6ce-2551-4e71-9884-576cd0503cbe'], 'holes': []}
{'token': '032217d5-1158-42a5-bc75-4d2074202554', 'polygon_token': 'e6b3072a-5686-43f2-aea0-a13af4f68f94', 'is_intersection': True, 'drivable_area_token': 'c3e28556-b711-4581-9970-b66166fb907d', 'exterior_node_tokens': ['961380bc-9006-4ad7-a6d9-7ed05e517be1', 'b05d14cf-9f26-49b1-8117-a146db7069d7', '31ca48e2-5078-4092-abe2-4d1ccf12c4e1', '3eddc1ad-158b-48fb-b7cf-987cba3db077', '2778ff68-967c-43f5-b205-aefed98737e9', 'a720f2f4-846e-4813-b0bc-022fac51403c', 'b1dcf58c-271f-4ae1-ade5-

{'drivable_area': [],
 'road_segment': [array([[ 529.74200174, 1286.62100877],
         [ 524.68039532, 1285.88309182],
         [ 524.96498787, 1283.39937793],
         [ 524.9372002 , 1281.54436788],
         [ 524.78527843, 1280.01693717],
         [ 524.50272488, 1279.62051201],
         [ 530.76939933, 1280.44420715]]),
  array([[ 532.42026507, 1159.261012  ],
         [ 530.18221851, 1164.11235435],
         [ 529.95904365, 1163.81478833],
         [ 528.1574112 , 1167.94173999],
         [ 528.54053339, 1167.88032375],
         [ 526.70640561, 1173.10115673],
         [ 526.23239463, 1173.53738083],
         [ 525.93557838, 1174.12789617],
         [ 522.66708022, 1173.23761426],
         [ 525.60243046, 1165.39205553],
         [ 529.14108272, 1156.87111734],
         [ 529.31524585, 1156.49796053],
         [ 532.2932746 , 1157.6120477 ],
         [ 532.2434982 , 1157.70854831],
         [ 532.37631515, 1159.1664641 ]]),
  array([[ 565.12037845, 1691.16671089],
         [ 563.

# Data Analysis

In [ ]:
info = []

for location in tqdm(locations[1:]):
    print(location)
    nusc_map = nusc_location[location]
    map_size = nusc_map.get_size()
    
    for x in tqdm(np.arange(100, map_size[0] + 200, 200)):
        for y in tqdm(np.arange(100, map_size[0] + 200, 200)):
            objects_list = nusc_map.get_closest_layers(nusc_map.non_geometric_layers, (x, y))
            for layer in nusc_map.non_geometric_layers:
                objects = objects_list[layer]
                for object_ in objects:
                    info.append([location, layer, x, y, object_.shape[0]])
                    
df = pd.DataFrame(info, columns = ['location', 'layer', 'map_x', 'map_y', 'number_of_objects'])

In [ ]:
print(nusc_map.non_geometric_polygon_layers)
print(nusc_map.non_geometric_line_layers)
print(nusc_map.non_geometric_layers)
layer

In [ ]:
df

In [ ]:
df = pd.DataFrame(info, columns = ['location', 'layer', 'map_x', 'map_y', 'number_of_objects'])

In [ ]:
df